In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import ModelCheckpoint

import numpy as np
from PIL import Image
import os 
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.client import device_lib

Using TensorFlow backend.


In [5]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8758661603669243413, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7641398477
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3950975244283840479
 physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [6]:
## Data loading 

trainData = "C:/Users/mostafaosama2/Desktop/autoenctrain/train"
testData = "C:/Users/mostafaosama2/Desktop/autoenctrain/test"

new_train = []
new_test = []

for filename in os.listdir(trainData):
    if filename.endswith('.tif'):
        image = Image.open(os.path.join(trainData, filename)) 
        new_train.append(np.asarray( image, dtype="uint8" ))

for filename in os.listdir(testData):
    if filename.endswith('.tif'):
        image = Image.open(os.path.join(testData, filename)) 
        new_test.append(np.asarray( image, dtype="uint8" ))

In [7]:
## Data preprocessing

# normalize all values between 0 and 1 and flatten 96x96 images into vectors of size 9216
# Basically switch from 2D matrices of 96 * 96 (and type int8) into 1D arrays of 9216 (and type float32)

x_train = np.asarray(new_train)
x_test = np.asarray(new_test)
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


27648

In [8]:
## Check data shapes and dimensions here

x_test.shape
x_train.shape

(9999, 96, 96, 3)

filters: the number of output filters in the convolution

kernel size: (3,3) means 3 x 3 kernel

stride: number of columns to move the kernel by on each convolution (1 is default)

UpSampling2D(pool_size=(2,2))

In [27]:
input_img = Input(shape=(96, 96, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) # 96 x 96 x 32
x = MaxPooling2D((3, 3), padding='same')(x) # 32 x 32 x 32
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x) # 32 x 32 x 64
x = MaxPooling2D((2, 2), padding='same')(x) # 16 x 16 x 64
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x) # 16 x 16 x 128 (small)
encoded = MaxPooling2D((2, 2), padding='same')(x) # 8 x 8 x 128

# at this point the representation is (8, 8, 128) 

x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded) # 8 x 8 x 128
x = UpSampling2D((2, 2))(x) # 16 x 16 x 128
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x) # 16 x 16 x 64
x = UpSampling2D((2, 2))(x) # 32 x 32 x 64
# x = Conv2D(32, (3, 3), activation='relu')(x)
x = UpSampling2D((3, 3))(x) # 96 x 96 x 64
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x) # 96 x 96 x 3

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='sgd', loss='mean_squared_error')
filepath = "C:/Users/mostafaosama2/Desktop/notebooks/checkpoint.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# autoencoder.summary()

In [28]:
n_epochs = 20
with tf.device('/GPU:0'):
    autoencoder_train = autoencoder.fit(x_train, x_train,
                    epochs=n_epochs,
                    batch_size=32,
                    shuffle=True,
                    validation_data=(x_test, x_test), 
                    callbacks=callbacks_list
                   )

Train on 9999 samples, validate on 1500 samples
Epoch 1/20
9999/9999 [==============================] - 9s 915us/step - loss: 0.0647 - val_loss: 0.0601

Epoch 00001: val_loss improved from inf to 0.06013, saving model to C:/Users/mostafaosama2/Desktop/notebooks/checkpoint.h5
Epoch 2/20
9999/9999 [==============================] - 9s 888us/step - loss: 0.0581 - val_loss: 0.0580

Epoch 00002: val_loss improved from 0.06013 to 0.05798, saving model to C:/Users/mostafaosama2/Desktop/notebooks/checkpoint.h5
Epoch 3/20
9999/9999 [==============================] - 9s 883us/step - loss: 0.0556 - val_loss: 0.0547

Epoch 00003: val_loss improved from 0.05798 to 0.05471, saving model to C:/Users/mostafaosama2/Desktop/notebooks/checkpoint.h5
Epoch 4/20
9999/9999 [==============================] - 9s 885us/step - loss: 0.0507 - val_loss: 0.0482

Epoch 00004: val_loss improved from 0.05471 to 0.04817, saving model to C:/Users/mostafaosama2/Desktop/notebooks/checkpoint.h5
Epoch 5/20
3104/9999 [======

KeyboardInterrupt: 

In [14]:
autoencoder.save('autoencoder_train.h5')

In [15]:
loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']
epochs = range(n_epochs)
plt.figure()
plt.plot(n_epochs, loss, 'bo', label='Training loss')
plt.plot(n_epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

NameError: name 'autoencoder_train' is not defined

In [ ]:
decoded_imgs = autoencoder.predict(x_test)
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(96, 96,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(96, 96,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()